In [12]:
import pickle
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error,accuracy_score,confusion_matrix
import numpy as np
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge

In [13]:
main_dataset= pickle.load(open('dataset/main_dataset.pkl','rb'))
main_dataset.head()

,call_id,agent_id_x,agent_tone,customer_tone,average_sentiment,silence_percent_average,customer_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,primary_call_reason,customer_name,elite_level_code,handle_time,allocation_time,hour,agent_tone_score,customer_tone_score
0,4667960400,963118,neutral,angry,-0.04,0.39,2033123310,2024-07-31 23:56:00,2024-08-01 00:03:00,2024-08-01 00:34:00,voluntary cancel,Matthew Foster,4.0,1860.0,420.0,23,3,1
1,1122072124,519057,calm,neutral,0.02,0.35,8186702651,2024-08-01 00:03:00,2024-08-01 00:06:00,2024-08-01 00:18:00,booking,Tammy Walters,0.0,720.0,180.0,0,4,3
2,6834291559,158319,neutral,polite,-0.13,0.32,2416856629,2024-07-31 23:59:00,2024-08-01 00:07:00,2024-08-01 00:26:00,irrops,Jeffery Dixon,0.0,1140.0,480.0,23,3,5
3,2266439882,488324,neutral,frustrated,-0.20,0.20,1154544516,2024-08-01 00:05:00,2024-08-01 00:10:00,2024-08-01 00:17:00,upgrade,David Wilkins,2.0,420.0,300.0,0,3,2
4,1211603231,721730,neutral,polite,-0.05,0.35,5214456437,2024-08-01 00:04:00,2024-08-01 00:14:00,2024-08-01 00:23:00,seating,Elizabeth Daniels,0.0,540.0,600.0,0,3,5


In [14]:
agent_data= pickle.load(open('dataset/agent_data.pkl','rb'))
agent_data.head()

,agent_id_x,total_calls,avg_tone_rating,avg_handle_time,avg_silence_percentage,avg_call_sentiment_score
0,102574,2,3.500000,3600.000000,0.260000,-0.075000
1,103651,298,3.278523,747.382550,0.293389,-0.027953
2,106000,113,3.292035,689.203540,0.274956,-0.032124
3,107876,6,3.166667,990.000000,0.405000,-0.065000
4,112609,11,3.454545,850.909091,0.286364,-0.020909


In [15]:
# Encoding categorical variables
# Dummy encoding for 'primary_call_reason'
main_dataset = pd.get_dummies(main_dataset, columns=['primary_call_reason','customer_tone'], drop_first=True)

# Check the new column names
print(main_dataset.columns)

# Define features and target, replace 
# 'primary_call_reason' with its dummy variable columns
X = main_dataset[['elite_level_code', 'hour'] + [col for col in main_dataset.columns if col.startswith('primary_call_reason_')]+[col1 for col1 in main_dataset.columns if col1.startswith('customer_tone_')]]  # Include all dummy columns for primary_call_reason
y = main_dataset['handle_time']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the model
random_forest = RandomForestRegressor(n_estimators=100, random_state=42)
gradient_boosting = GradientBoostingRegressor(n_estimators=100, random_state=42)
xgboost = XGBRegressor(n_estimators=100, random_state=42)

stacking_model = StackingRegressor(
    estimators=[
        ('rf', random_forest),
        ('gb', gradient_boosting),
        ('xgb', xgboost)
    ],
    final_estimator=Ridge()
)

stacking_model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = stacking_model.predict(X_test)
print("Prediction: ", y_pred)
print("Actual: ", y_test.values)

# Calculate Mean Squared Error and Root Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error: {rmse}')
print(f'Mean Squared Error: {mse}')

# Create a DataFrame to compare actual and predicted values
comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

# Display the DataFrame
print("-------------------------------")
print( comparison_df)


# Feature importance (optional)
# importances = model.feature_importances_
# indices = importances.argsort()[::-1]

# Display feature importance
# for f in range(X.shape[1]):
#     print(f'{f + 1}. Feature {X.columns[indices[f]]} ({importances[indices[f]]})')


Index(['call_id', 'agent_id_x', 'agent_tone', 'average_sentiment',
       'silence_percent_average', 'customer_id', 'call_start_datetime',
       'agent_assigned_datetime', 'call_end_datetime', 'customer_name',
       'elite_level_code', 'handle_time', 'allocation_time', 'hour',
       'agent_tone_score', 'customer_tone_score',
       'primary_call_reason_booking', 'primary_call_reason_check in',
       'primary_call_reason_check-in', 'primary_call_reason_checkout',
       'primary_call_reason_communications',
       'primary_call_reason_digital support', 'primary_call_reason_disability',
       'primary_call_reason_etc', 'primary_call_reason_irrops',
       'primary_call_reason_mileage plus', 'primary_call_reason_other topics',
       'primary_call_reason_post flight', 'primary_call_reason_post-flight',
       'primary_call_reason_products & services',
       'primary_call_reason_products and services',
       'primary_call_reason_schedule change', 'primary_call_reason_seating',
     